# Zadanie 1
Jako przykład rozważmy dane zawierające poziomy przepływu ($\mu g/l$) w trzech grupach pacjentów z zastawką serca z różnymi poziomami wentylacji azotem. W analizie wzięło udział 22 osoby (https://github.com/przem85/statistics/blob/master/D7/ANOVA1.txt).

Zerową hipoteza dla ANOVA mówi, że wszystkie grupy pochodzą z tej samej populacji. 


In [2]:
import numpy as np
import scipy.stats as stats
import pandas as pd

In [3]:
data = np.loadtxt('ANOVA1.txt')
 # Sort them into groups, according to column 1
group1 = data[data[:,1]==1,0]
group2 = data[data[:,1]==2,0]
group3 = data[data[:,1]==3,0]

OSError: ANOVA1.txt not found.

In [16]:
data.shape

(22, 2)

## Sprawdzamy założenie o równości wariancji 

In [17]:
# First, check if the variances are equal, with the "Levene"-test
(W,p) = stats.levene(group1, group2, group3)
if p<0.05:
    print(('Warning: the p-value of the Levene test is <0.05: p={0}'.format(p)))

## Wykonujemy ANOVE jednoczynnikową 

In [18]:
# Do the one-way ANOVA
F_statistic, pVal = stats.f_oneway(group1, group2, group3)

In [19]:
print((F_statistic, pVal))
if pVal < 0.05:
    print('One of the groups is significantly different.')

(3.7113359882669763, 0.043589334959178244)
One of the groups is significantly different.


## Bardziej szczegółowe wyniki ANOVA można otrzymać za pomocą modelowania statystycznego:

In [20]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
# Elegant alternative implementation, with pandas & statsmodels
df = pd.DataFrame(data, columns=['value', 'treatment'])    
model = ols('value ~ C(treatment)', df).fit()
anovaResults = anova_lm(model)
print(anovaResults)

                df        sum_sq      mean_sq         F    PR(>F)
C(treatment)   2.0  15515.766414  7757.883207  3.711336  0.043589
Residual      19.0  39716.097222  2090.320906       NaN       NaN


# Możemy policzyć ten test na piechotę

In [21]:
data = np.loadtxt('ANOVA1.txt')

In [22]:
# Convert them to pandas-forman and group them by their group value
df = pd.DataFrame(data, columns=['values', 'group'])
groups = df.groupby('group')

Najpierw oblicza się Sums of squares ($SS_{Total}$). Otrzymujemy 

$$
SS_{Treatments}=15 515.76
$$ 

oraz 

$$
SS_{Error}=39.716.09.
$$

In [23]:
# The "total sum-square" is the squared deviation from the mean
ss_total = np.sum((df['values']-df['values'].mean())**2)
ss_total

55231.86363636364

In [24]:
# Calculate ss_treatment and  ss_error
(ss_treatments, ss_error) = (0, 0)
for val, group in groups:
    ss_error += sum((group['values'] - group['values'].mean())**2)
    ss_treatments += len(group) * (group['values'].mean() - df['values'].mean())**2
print(ss_treatments)
print(ss_error)

15515.766414141408
39716.0972222


Średnie kwadraty (mean squares) (MS) to SS podzielone przez odpowiednie stopnie swobody (DF).

In [25]:
df_groups = len(groups)-1
df_residuals = len(data)-len(groups)

Otrzymujemy również wartość statystyki $F$:
$$
F=\frac{MS_{Treatments}}{MS_{Error}}=\frac{SS_{Treatments}/(n_{groups}-1)}{MS_{Error}/(n_{total}-n_{groups})}
$$

In [26]:
F = (ss_treatments/df_groups) / (ss_error/df_residuals)
df = stats.f(df_groups,df_residuals)
p = df.sf(F)
print(('ANOVA-Results: F = {0}, and p<{1}'.format(F, p)))

ANOVA-Results: F = 3.71133598827, and p<0.0435893349592
